In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import re
import keras
import random
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.applications import DenseNet121,Xception, DenseNet201


In [ ]:
import numpy as np
import h5py

hf = h5py.File('../input/breastnew4k/Train224x224xnew.h5', 'r')
hf.keys()

x = np.array(hf.get('x'))
y = np.array(hf.get('y'))


labels = np.load('../input/breastcancer225x500/Cancerlabels.npy')

hf.close()

del hf

In [ ]:
print(labels)

In [ ]:
#NewVer
import numpy as np
import h5py

hf_train = h5py.File('../input/1234456/Train1.h5', 'r')
hf_train.keys()
hf_test = h5py.File('../input/cogangthemchutnuaaaaa/Valid1.h5', 'r')
hf_test.keys()
labels = np.load('../input/breastcancer225x500/Cancerlabels.npy')
x_train = np.array(hf_train.get('x'))
y_train = np.array(hf_train.get('y'))
x_test = np.array(hf_test.get('x'))
y_test = np.array(hf_test.get('y'))
hf_train.close()
hf_test.close()

del hf_train, hf_test

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x , y , test_size = 0.2 , stratify = y , random_state = 0)

In [ ]:
del x,y

In [ ]:
del  model

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
with strategy.scope():
    pre_trained_model = DenseNet121(input_shape=(224,224,3),include_top=False, weights="imagenet")
    
#     for layer in pre_trained_model.layers[:15]:
        #layer.trainable = False
    
    model = Sequential([
        pre_trained_model,
        MaxPool2D((2,2) , strides = 2),
        Flatten(),
        Dense(4 , activation='softmax')])
    model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
print(len(x_train),len(x_test),len(y_test),len(y_train))

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.3, min_lr=0.000001)

In [ ]:
print(learning_rate_reduction)

In [ ]:
history = model.fit(x_train,y_train, batch_size = 64 , epochs =64  , validation_data = (x_test, y_test),callbacks = [learning_rate_reduction])

In [ ]:
model.save("Densenet201.h5")

In [ ]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
history.history['val_loss']

   


In [ ]:
epochs = [i for i in range(64)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']

train_loss = history.history['loss']

val_acc = history.history['loss']

val_loss = history.history['val_loss']

fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Valid Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Valid Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:

def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(model_history.history[acc][2:63]) + 1), model_history.history[acc][2:63])
    axs[0].plot(range(1, len(model_history.history[val_acc][2:63]) + 1), model_history.history[val_acc][2:63])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history[acc][2:63]) + 1), len(model_history.history[acc][2:63]) / 10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1, len(model_history.history['loss'][2:63]) + 1), model_history.history['loss'][2:63])
    axs[1].plot(range(1, len(model_history.history['val_loss'][2:63]) + 1), model_history.history['val_loss'][2:63])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_history.history['loss'][2:63]) + 1), len(model_history.history['loss'][2:63]) / 10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
    plt.savefig('roc.png')
 
plot_model_history(history)

In [ ]:
predictions = model.predict_classes(x_test)
predictions[:4]

In [ ]:
y_test_inv = label_binarizer.inverse_transform(y_test)

In [ ]:
print(classification_report(y_test_inv, predictions, target_names = labels))

In [ ]:
cm = confusion_matrix(y_test_inv,predictions)

In [ ]:
cm = pd.DataFrame(cm , index = labels , columns = labels)
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Reds", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = labels , yticklabels = labels)

In [ ]:
model.save('Xeption_new.h5')

In [ ]:
model1 = tf.keras.models.load_model('../input/tlcn-model/BreastCancer224.h5')

model1.summary()

In [ ]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
epochs = [i for i in range(50)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
y_test_inv = label_binarizer.inverse_transform(y_test)

In [ ]:
print(classification_report(y_test_inv, predictions, target_names = labels))

In [ ]:
cm = confusion_matrix(y_test_inv,predictions)
cm

In [ ]:
cm = pd.DataFrame(cm , index = labels , columns = labels)

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = labels , yticklabels = labels)

In [ ]:
tf.keras.experimental.export_saved_model(model, 'model.h5')

In [ ]:
def predict(path):
#path ='../input/asdfghjkl/begin.png'
    img = cv2.imread(path)
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()
    
    img_size=225
    img = cv2.resize(img,(img_size,img_size),3)
    img = np.array(img) / 255
    img = np.array(img)
    img = img.reshape(-1, img_size, img_size, 3)
    # Generate predictions for samples
    predictions = model.predict(img)
    return predictions

In [ ]:
a = predict(r'../input/testdata/test37.tif')

